In [99]:
data = !ls data/group_a/HKG*.csv

In [100]:
data

['data/group_a/HKG201409.csv',
 'data/group_a/HKG201410.csv',
 'data/group_a/HKG201411.csv',
 'data/group_a/HKG201412.csv',
 'data/group_a/HKG201501.csv']

# Treatment vs Control

In [101]:
import pandas as pd

In [102]:
# Treatment vs Control
treatment = pd.read_excel("treatment/treatment_groupHK.xlsx")

In [103]:
treatment_code = treatment['证券代码'].tolist()

# Processing

In [104]:
df = pd.DataFrame(columns=pd.read_csv(data[0], nrows=1).columns.tolist())

In [105]:
df = pd.read_csv(data[0])

In [106]:
for i in range(len(data)):
    df_next = pd.read_csv(data[i])
    df_next = df_next.dropna()
    df_next = df_next[(df_next != 0).all(1)]
    df = df.append(df_next, ignore_index=True)

In [107]:
df = df.dropna()

In [108]:
df['treatdum'] = df['stock'].apply(lambda x: 1 if int(x[:-3]) in treatment_code else 0)

In [109]:
df['date'] = pd.to_datetime(df['date'])

In [110]:
df.loc[df['date']<=pd.datetime(2014,11,17), 'perioddum'] = 0
df.loc[df['date']>pd.datetime(2014,11,17), 'perioddum'] = 1
df['did_dum'] = df['perioddum']*df['treatdum']

<ipython-input-110-0f47a2e734f0>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date']<=pd.datetime(2014,11,17), 'perioddum'] = 0
<ipython-input-110-0f47a2e734f0>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date']>pd.datetime(2014,11,17), 'perioddum'] = 1


# Processing

In [111]:
df_treatment = df[df['treatdum'] == 1]
df_control = df[df['treatdum'] == 0]

In [112]:
df_control = df_control[df_control["quote2trade"] <= 10]

In [113]:
df_treatment_mean = df_treatment.groupby("date").mean().reset_index()
df_control_mean = df_control.groupby("date").mean().reset_index()

# Visualization

In [117]:
import pandas as pd
from highcharts import Highchart, Highstock
import csv
import datetime

OPTIONS_DEFAULT = {
                  'chart': {'zoomType': 'x'},
                  'navigator': {'enabled': False},
                  'rangeSelector': {'enabled': False},
                  'legend': {'enabled': True,
                             'itemStyle':{'fontWeight':'normal'}},
                  'title': {"text":"HKG Treatment vs Control: Trade Value"}
                    }

options_line = OPTIONS_DEFAULT
options_line['yAxis'] = [{'opposite': False, 'title':{'text': 'Trade Value'},'tickAmount':8 }]

Line_Chart = Highstock()

def plotline(date, text):
    line = {
                'color': '#c8c8c8',
                'width': 1,
                'value': ((date -  datetime.datetime.utcfromtimestamp(0)).total_seconds() * 1000.0), 
                'zIndex': 1,
                'label': {'text': text,
                            'style': {
                            'color': 'grey'},
                            'rotation': 90,
                            'align': 'center',
                            'x': 3,
                            'y': 50}
            }
    return line

event_dict = {'2014-11-17': 'Stock Connect'}

for date in event_dict:
    date_object = datetime.datetime.strptime(date, '%Y-%m-%d')
    line = plotline(date_object, event_dict[date])
    try:
        options_line['xAxis']['plotLines'].append(line)
    except KeyError:
        options_line['xAxis']={'plotLines': [line]}

Line_Chart.set_dict_options(options_line)

Line_Chart.add_data_set(df_treatment_mean.filter(['date','trade_value']).values.tolist(), name='treatment', yAxis=0)
Line_Chart.add_data_set(df_control_mean.filter(['date','trade_value']).values.tolist(), name='control', yAxis=0)

Line_Chart

# DID

In [59]:
import pandas as pd
from linearmodels.panel import PanelOLS

df.loc[df['date-time']<=pd.datetime(2014,11,17), 'perioddum'] = 0
df.loc[df['date-time']>pd.datetime(2014,11,17), 'perioddum'] = 1
df['did_dum'] = df['perioddum']*df['treatdum']

## set index if you need to identify entity_effects/time_effects
df = df.set_index(['date-time'])

## Now run the DID estimation
x_variables = ['perioddum', 'treatdum', 'did_dum']
y_variable = 'trade_volume'

regression=PanelOLS(df[y_variable], df[x_variables], entity_effects=True, drop_absorbed=True)
regression.fit()

<ipython-input-59-3051c18f260a>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date-time']<=pd.datetime(2014,11,17), 'perioddum'] = 0
<ipython-input-59-3051c18f260a>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.loc[df['date-time']>pd.datetime(2014,11,17), 'perioddum'] = 1


ValueError: Series can only be used with a 2-level MultiIndex